# Recommender Systems Demo

This notebook provides a demonstration of different types of recommender systems:

1. **Popularity-Based Recommendation**
2. **Content-Based Filtering**
3. **Collaborative Filtering**
4. **Matrix Factorization (SVD)**
5. **Hybrid Approaches**

For each method, we describe the packages used, important functions and hyperparameters, and demonstrate how to evaluate each model.

## 1. Popularity-Based Recommendation
This method recommends the most popular items.

### Packages Used:
- `pandas`

### Evaluation:
- Not personalized, mainly used as a baseline.

In [ ]:
import pandas as pd
import numpy as np

# Sample dataset
ratings = pd.DataFrame({
    'user_id': [1, 1, 2, 2, 3, 3, 4],
    'item_id': ['A', 'B', 'A', 'C', 'B', 'D', 'D'],
    'rating': [5, 4, 5, 3, 2, 5, 4]
})

# Most popular items
popularity = ratings.groupby('item_id')['rating'].mean().sort_values(ascending=False)
popularity

## 2. Content-Based Filtering
This method uses item features (e.g., genres) to recommend similar items.

### Packages Used:
- `sklearn` for `TfidfVectorizer`, `cosine_similarity`

### Key Functions and Hyperparameters:
- `TfidfVectorizer()`: max_features, stop_words

### Evaluation:
- Precision@K, Recall@K

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Example item content
items = pd.DataFrame({
    'item_id': ['A', 'B', 'C', 'D'],
    'description': ["Sci-fi action", "Romantic comedy", "Drama", "Action thriller"]
})

# TF-IDF + cosine similarity
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(items['description'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

pd.DataFrame(cos_sim, index=items['item_id'], columns=items['item_id'])

## 3. Collaborative Filtering (User-User)

This method recommends items liked by similar users.

### Packages:
- `sklearn`: `cosine_similarity`

### Evaluation:
- RMSE, MAE
- Precision@K

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Create user-item matrix
user_item = ratings.pivot(index='user_id', columns='item_id', values='rating').fillna(0)
sim_matrix = cosine_similarity(user_item)
pd.DataFrame(sim_matrix, index=user_item.index, columns=user_item.index)

## 4. Matrix Factorization (SVD)

### Packages:
- `surprise`: a specialized package for recommender systems

### Key Functions:
- `SVD` algorithm from `surprise`
- `Dataset.load_from_df`, `cross_validate`

### Hyperparameters:
- `n_factors`, `n_epochs`, `lr_all`, `reg_all`

### Evaluation:
- RMSE, MAE

In [ ]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['user_id', 'item_id', 'rating']], reader)

model = SVD(n_factors=50, n_epochs=20, lr_all=0.005, reg_all=0.02)
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

## 5. Hybrid Approach

Combine content and collaborative filtering.

### Strategy:
- Weighted sum of normalized content similarity and user similarity matrices

### Evaluation:
- Custom depending on the fusion method
- Precision@K, Recall@K